In [2]:
import tqdm
from tqdm import tqdm
import pandas as pd 
import random
import json
import spacy
import translators as ts
import scipy
import pickle
import numpy as np

df = pd.read_excel('../Excel_files/Full_Poem_Dataset_12-17.xlsx')
df = df.drop(columns=['Unnamed: 0'])
records = df.to_dict('records')
random.choice(records)

Using United States server backend.


{'Text': '* * * *\nЯ сдерживал себя в чистилище, в метро и\nв хрустальной башне с огненной водой,\nкогда я умирал — меня держали трое:\nотец и сын, мятежный дух святой.\n\nИ если ты — любовь, побудь со мной немножко,\nзайди на порнохаб — поставь за всех свечу,\nя — кофе или чай, собака или кошка,\nя сдерживал себя, но больше — не хочу.\n\nБетховен или бах, чайковский или верди,\nхранитель белых слов, вершитель черных дел,\nи не благодари меня за опыт смерти:\nактёр играл врача и вскоре — заболел.\n\nВ чём логика, когда — мы говорим о чуде,\nзвучит бессвязный свет из флейты золотой,\nя — память о вине в раздвоенном сосуде,\nкоторый — переполнен пустотой.\n\nЕщё я состою из самых чёрствых крошек —\nсмахни меня в ладонь, спасая и губя,\nя мог не воскрешать: детей, собак и кошек,\nя склеил старый мир и сдерживал себя.\n\nСмотрю на пикассо, читаю, бл@дь, неруду,\nопять мятежный дух витает над толпой:\nи всякий — устрашись, когда я снова буду —\nединственным собой, единственным тобой.\n\n202

In [3]:
authorsDf = pd.read_excel('../Excel_files/Thesis Authors.xlsx')
authorsDf = authorsDf.fillna('N/A')
aRecs = authorsDf.to_dict('records')
authorDict = dict()
for a in aRecs:
    authorDict[a['Author']] = a
authorsDf.head()

,Number of Poems,Author,City,Country,Identification,Notes,Link to bio
0,3,Виген Аракелян,Razdan,Armenia,N/A,N/A,https://polutona.ru/?show=arakelian
1,1,Герман Лукомников,Baku,Azerbaijan,N/A,N/A,https://ru.wikipedia.org/wiki/%D0%9B%D1%83%D0%...
2,1,Александр Иличевский,Sumgait,Azerbaijan,N/A,N/A,https://ru.wikipedia.org/wiki/%D0%98%D0%BB%D0%...
3,7,Леонид Шваб,Babruysk,Belarus,N/A,N/A,http://www.litkarta.ru/world/israel/persons/sh...
4,6,Ирина Валерина,Babruysk,Belarus,N/A,N/A,N/A


In [4]:
with open(f'Thesis_Authors16.json', 'w', encoding='utf-8') as f:
    json.dump(aRecs, f, ensure_ascii=False, indent=4)

In [5]:
authorsDf.to_json('Thesis_Authors.json')

In [6]:
authorDict['Игорь Сатановский']

{'Number of Poems': 5,
 'Author': 'Игорь Сатановский',
 'City': 'Kyiv',
 'Country': 'Ukraine',
 'Identification': 'N/A',
 'Notes': '1969, lived in usa since 1989',
 'Link to bio': 'http://www.litkarta.ru/world/usa/persons/satanovsky-i/'}

In [7]:
'aaaaaasdfsdf'.count('df')

2

In [8]:
### Find repetition
# for rec in records:
#     if rec['Text'].lower().count('война') > 5:
#         print(rec['UniqueIndex'])
#         print(rec['Text'])
#         print()
#         print()
rec = [r for r in records if r['UniqueIndex'] == 380][0]
print(rec)

{'Text': '* * *\n\neслибыеслибыеслибыеслибы\nеслибыеслибыеслепыеслепы\nеслепыеслепыеслепыеслепы\nеслепыеслепыеслибыеслибы\neслибыеслибыеслибыеслибы\n\n**Пушкин и война**\n\nВойна идёт, а Пушкин пишет \nВойна идёт, а Пушкин пишет \nВойна идёт, а Пушкин пишет \nВойна идёт идёт идёт\n \nА Пушкин пишет пишет пишет \nА Пушкин пишет пишет пишет \nА Пушкин пишет пишет пишет \nВойна идёт идёт идёт', 'Author': 'Игорь Сатановский', 'Before or after': 'After', 'Source': 'ROAR V3', 'Date posted': 'None', 'UniqueIndex': 380}


In [16]:
# with open('textDict.pickle', 'rb') as handle:
#     textDict = pickle.load(handle)
# textDict['двадцать']

### Statistics for Methods section

In [9]:
nlp = spacy.load("ru_core_news_lg")

In [10]:
for rec in tqdm(records):
    doc = nlp(rec['Text'])
    rec['doc'] = doc

100%|███████████████████████████████████████| 3222/3222 [03:44<00:00, 14.35it/s]


In [13]:
# look at lemma counts
tokenCt = dict()
poemCt = dict()
tokenLemmas = dict()

for p in ['Before','After']:
    tokenCt.setdefault(p, 0)
    poemCt.setdefault(p, 0)
    tokenLemmas.setdefault(p, set())
for rec in tqdm(records):
    poemCt[rec['Before or after']] += 1

    for t in rec['doc']:
        tokenCt[rec['Before or after']] += 1 
        tokenLemmas[p].add(t.lemma_)
        
print(tokenCt)
print(poemCt)
print(len(tokenLemmas['Before']),len(tokenLemmas['After']))

100%|█████████████████████████████████████| 3222/3222 [00:00<00:00, 4979.36it/s]

{'Before': 291769, 'After': 266511}
{'Before': 1661, 'After': 1561}
0 43565


In [12]:
tokenLemmas['Before']

set()

In [104]:
### Tense breakdown each time period
tenseDict = {'Before' : dict(), 'After' : dict()}
for p in tenseDict:
    for t in ['Past','Pres','Fut']:
        tenseDict[p][t] = 0
for r in tqdm(records):
    p = r['Before or after']
    for t in r['doc']:
        if t.pos_ == 'VERB' and 'Tense' in t.morph.to_dict():
            tense = t.morph.to_dict()['Tense']
            m = t.morph
            tenseDict[p].setdefault(tense, 0)
            tenseDict[p][tense]+=1
            

100%|█████████████████████████████████████| 3222/3222 [00:01<00:00, 3187.59it/s]


In [101]:
for p in tenseDict:
    print(p)
    for t in tenseDict[p]:
        print(t,tenseDict[p][t]/(tenseDict[p]['Past']+tenseDict[p]['Pres']+tenseDict[p]['Fut']))
    print()

Before
Past 0.4044048395276389
Pres 0.5074983699195827
Fut 0.08809679055277839

After
Past 0.4118027224623339
Pres 0.48103566668013087
Fut 0.10716161085753524



In [113]:
a1 = tenseDict['Before']['Pres']*[1]+ tenseDict["Before"]['Past']*[0] + tenseDict['Before']['Fut']*[0]
a2 = tenseDict['After']['Pres']*[1]+ tenseDict["After"]['Past']*[0] + tenseDict['After']['Fut']*[0]
test = scipy.stats.ttest_ind(a1, a2, axis=0, equal_var=True, nan_policy='propagate', 
                          permutations=None, random_state=None, alternative='two-sided', trim=0)
print(test)

Ttest_indResult(statistic=6.0487816625211215, pvalue=1.4693138352414718e-09)


In [114]:
# fut : Ttest_indResult(statistic=-7.359306571050845, pvalue=1.8756963438929145e-13)
# Pres: Ttest_indResult(statistic=6.0487816625211215, pvalue=1.4693138352414718e-09)

#### Calculating two sample one sided t-test

In [14]:
textDict = dict()
lemmaCts = dict()
totalPoemCts = dict()
scipyArray = dict()
countryBreakdowns = dict()

for r in tqdm(records):
    p = r['Before or after']
    totalPoemCts.setdefault(p, 0)
    lemmasFound = set()
    totalPoemCts[p] += 1
    for t in r['doc']:
        if t.lemma_ not in lemmasFound:
            # lemmaCts.setdefault(t.lemma_, {'Before' : 0, 'After' : 0})
            lemmaCts.setdefault(t.lemma_, {'Before' : 0, 'After' : 0, 'nlpToken' : t})
            lemmaCts[t.lemma_][p] += 1
            
            textDict.setdefault(t.lemma_, {'Before' : [], 'After' : []})
            textDict[t.lemma_][p].append({'ID' : r['UniqueIndex'], 'tokenText' : t.text})
            
            countryBreakdowns.setdefault(t.lemma_, {'Before' : dict(), 'After' : dict()})
            try:
                country = authorDict[r['Author']]['Country']
            except:
                country = 'Unknown'
            if pd.isna(country):
                country = 'Unknown'
                
            countryBreakdowns[t.lemma_][p].setdefault(country, 0)
            countryBreakdowns[t.lemma_][p][country] += 1
            
            lemmasFound.add(t.lemma_)
        else:
            continue
lemmaFreqs = dict()
for word in lemmaCts:
    lemmaFreqs.setdefault(word, {'Before' : 0, 'After' : 0})
    for p in ['Before','After']:
        lemmaFreqs[word][p] = lemmaCts[word][p]/totalPoemCts[p]
        lemmaFreqs[word][p+'Ct'] = lemmaCts[word][p]
        lemmaFreqs[word]['nlp'] = lemmaCts[word]['nlpToken']
        
# create arrays for scipy
scipyArray = dict()
for lem in lemmaCts:
    scipyArray.setdefault(lem, {'Before' : [], 'After' : []})

for r in tqdm(records):
    p = r['Before or after']
    recLemmas = set([t.lemma_ for t in r['doc']])
    for rl in recLemmas:
        scipyArray[rl][p].append(1)
    for lem in lemmaCts:
        if lem not in recLemmas:
            scipyArray[lem][p].append(0)

100%|███████████████████████████████████████| 3222/3222 [00:55<00:00, 58.27it/s]


In [15]:
for l in tqdm(lemmaFreqs):
    for p in ['Before','After']:
        lemmaFreqs[l]['countryInfo'] = countryBreakdowns[l]

100%|█████████████████████████████████| 43565/43565 [00:00<00:00, 199301.13it/s]


In [16]:
lemmaFreqs['война']

{'Before': 0.059000602046959665,
 'After': 0.29276105060858426,
 'BeforeCt': 98,
 'nlp': война,
 'AfterCt': 457,
 'countryInfo': {'Before': {'Ukraine': 22,
   'Russia': 53,
   'Kazakhstan': 2,
   'Latvia': 1,
   'N/A': 9,
   'Belarus': 6,
   'Moldova': 4,
   'Uzbekistan': 1},
  'After': {'Russia': 190,
   'Ukraine': 126,
   'Moldova': 13,
   'N/A': 79,
   'Lithuania': 2,
   'Unknown': 1,
   'Belarus': 36,
   'Georgia': 2,
   'Latvia': 3,
   'Kazakhstan': 1,
   'Uzbekistan': 4}}}

In [48]:
# import pickle

# with open('textDict.pickle', 'wb') as handle:
#     pickle.dump(textDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
jsonData = []
theselemmas = set()
for keyword in tqdm(lemmaCts):
    if lemmaCts[keyword]['Before'] + lemmaCts[keyword]['After'] >= 25:
        theselemmas.add(keyword)
        a1 = scipyArray[keyword]['Before']
        a2 = scipyArray[keyword]['After']
        test = scipy.stats.ttest_ind(a1, a2, axis=0, equal_var=True, nan_policy='propagate', 
                              permutations=None, random_state=None, alternative='two-sided', trim=0)
        lemmaFreqs[keyword]['stats'] = {'statistic' : test.statistic, 'pvalue' : test.pvalue}
        jsonData.append({
            'keyword' : keyword,
            'info' : lemmaFreqs[keyword]
        })

100%|██████████████████████████████████| 43565/43565 [00:02<00:00, 17848.52it/s]


In [24]:
# with open(f'frequencyData2-3_1.json', 'w', encoding='utf-8') as f:
#     json.dump(jsonData, f, ensure_ascii=False, indent=4)

#### Significant word breakdown

In [18]:
# filter data
data = [d for d in jsonData if not d['info']['Before']*d['info']['After']==0 ]
data = [d for d in data if d['info']['stats']['pvalue'] < .05]
data = [d for d in data if d['keyword'].isalpha()]
irrelevantWords = ['авторский', 'блог', 'личный', 'источник','автор']
data = [d for d in data if d['keyword'] not in irrelevantWords]
print(len(jsonData), len(data))

1626 421


In [19]:
lemsForPCA = [d['keyword'] for d in data]
len(lemsForPCA)

421

In [20]:
# with open('word4map.txt', 'w') as f:
#     f.write('\n'.join(lemsForPCA))

In [21]:
posDict = {'increase' : dict(), 'decrease' : dict()}
for d in data:
    which = ''
    if d['info']['After'] - d['info']['Before'] > 0:
        which = 'increase'
    else:
        which = 'decrease'
    pos = d['info']['nlp'].pos_
    posDict[which].setdefault(pos, [])
    posDict[which][pos].append(d)

In [22]:
sorted([p for p in set(list(posDict['increase'].keys()) + list(posDict['decrease'].keys()))])

['ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CCONJ',
 'DET',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'SCONJ',
 'VERB']

In [71]:
for pos in posDict['increase']:
    for n in posDict['increase'][pos]:
        print(n['keyword'])

мир
эпоха
год
кровь
сын
надежда
век
взрыв
город
страна
ад
ужас
ребёнок
слава
список
беда
народ
день
смерть
войско
пепел
выстрел
приказ
бог
дом
карта
зло
весна
сотня
страх
вина
герой
солдат
буква
пуля
подвал
война
ложь
молитва
совесть
град
осколок
ракета
танк
мозг
стыд
февраль
вера
цель
март
корабль
туча
бой
чёрт
труп
металл
добро
победа
честь
стон
оружие
крест
ров
новость
марш
строй
враг
колонна
боль
фронт
вой
слёзы
село
мрак
сирена
убийца
самолёт
рана
армия
ненависть
крик
обломок
руина
бомба
очередь
снаряд
фотография
днём
лента
железо
действие
парад
сейчас
теперь
вперёд
скоро
также
нынче
ваш
наш
злой
чужой
страшный
военный
божий
вечный
русский
виноватый
российский
родный
святой
ядерный
весенний
мирный
кровавый
безумный
мировой
убивать
нет
писать
разрушить
воевать
прийти
начаться
убить
идти
выжить
орать
взять
надеяться
закончиться
молиться
ползти
выть
рыдать
плакать
спасти
победить
погибнуть
кричать
стрелять
ненавидеть
отправить
суметь
бомбить
мешать
прощать
послать
разорвать
рваться
ж

## Similarity

In [42]:
import time
import os.path

from django.conf import settings

from navec import Navec
from numpy import dot
from numpy.linalg import norm
from annoy import AnnoyIndex


NAVEC_PATH = "data4notebooks/navec_hudlit_v1_12B_500K_300d_100q.tar"
# ANNOY_INDEX_PATH = os.path.join(settings.ROOT_DIR, "parser_tool", "data", "ANNOY_tree.ann")

navec = Navec.load(NAVEC_PATH)
vocabulary = navec.vocab.words
word_to_index = dict()
for i, word in enumerate(vocabulary):
    word_to_index[word] = i

In [45]:
# LOAD JSON WITH AVERAGE VALS
with open(f'../../files2big/beforeandafter768D0', 'r', encoding='utf-8') as f:
    jsonVecs = json.load(f)

In [46]:
word2vec = dict()
for v in jsonVecs:
    if 'Average' in v['word']:
        word2vec[v['word'].replace('_Average','')] = v['vector']

In [67]:
nlp(word)[0].pos_

'NOUN'

In [61]:
vecDict = dict()
for direction in posDict:
    vecDict.setdefault(direction, dict())
    for pos in posDict[direction]:
        vecDict[direction].setdefault(pos, [])
        keywords = [d['keyword'] for d in posDict[direction][pos]]
        for word in tqdm(keywords):
            try:
                navector = navec[word].tolist()
            except:
                print('not in navec dictionary')
                navector = np.zeros(300).tolist()
            vecDict[direction][pos].append({
                "word" : word,
                "translatedword" : ts.google(word),
                "bertvector" : word2vec[word],
                'navecvector' : navector,
                'countryInfo' : countryBreakdowns[word]
            })

100%|█████████████████████████████████████████████| 2/2 [00:02<00:00,  1.03s/it]


In [ ]:
for 

In [62]:
with open(f'../significantPCA/sigMeanDiffVecsTranslated4.json', 'w', encoding='utf-8') as f:
    json.dump(vecDict, f, ensure_ascii=False, indent=4)

## Synsets

In [1]:
!ruwordnet download

downloading a ruwordnet model from https://github.com/avidale/python-ruwordnet/releases/download/0.0.4/ruwordnet-2021.db


In [2]:
from ruwordnet import RuWordNet
wn = RuWordNet()

In [11]:
lemma = 'мир'
for sense in wn.get_senses(lemma):
    print(sense.synset.title.lower())

вселенная, мироздание, мир
сообщество (совокупность людей)
мир, отсутствие конфликтов
мир, отсутствие войны


In [12]:
lemmaCts = dict()
totalPoemCts = dict()
scipyArray = dict()
sense2lemmas = dict()

for r in tqdm(records):
    p = r['Before or after']
    totalPoemCts.setdefault(p, 0)
    lemmasFound = set()
    totalPoemCts[p] += 1
    for t in r['doc']:
        if t.lemma_ not in lemmasFound:
            for sense in wn.get_senses(t.lemma_):
                title = sense.synset.title.lower()
                sense2lemmas.setdefault(sense, set())
                sense2lemmas[sense].add(t.lemma_)
                lemmaCts.setdefault(title, {'Before' : 0, 'After' : 0})
                lemmaCts[title][p] += 1
            lemmasFound.add(t.lemma_)
        else:
            continue
lemmaFreqs = dict()
for word in lemmaCts:
    lemmaFreqs.setdefault(word, {'Before' : 0, 'After' : 0})
    for p in lemmaCts[word]:
        lemmaFreqs[word][p] = lemmaCts[word][p]/totalPoemCts[p]
        lemmaFreqs[word][p+'Ct'] = lemmaCts[word][p]

NameError: name 'tqdm' is not defined

In [ ]:
    # check if title already added
    title = synset.title
    # change synset label to lemma label if only one lemma mentioned
    if len(synsetTokens[synset]) == 1:
        title = list(synsetTokens[synset].keys())[0]
        
    # if title already added, skip entry, else continue
    if title in titleAddedAlready:
        print('OLD:',title, synset.title)
        print('NEW:',title, lemma2synset[title])
        continue
    else:
        lemma2synset[title] = synset.title
        titleAddedAlready.add(title)

#### Language Detection

In [2]:
# !pip3 install spacy-language-detection

In [3]:
import spacy
from spacy.language import Language

from spacy_language_detection import LanguageDetector


def get_lang_detector(nlp, name):
    return LanguageDetector(seed=42)  # We use the seed 42


In [4]:

nlp_model = spacy.load("ru_core_news_lg")
Language.factory("language_detector", func=get_lang_detector)
nlp_model.add_pipe('language_detector', last=True)

# Sentence level language detection
text = "This is English text. Er lebt mit seinen Eltern und seiner Schwester in Berlin. Yo me divierto todos los días en el parque. Je m'appelle Angélica Summer, j'ai 12 ans et je suis canadienne."
doc = nlp_model(text)
for i, sent in enumerate(doc.sents):
    print(sent, sent._.language)

This is English text. {'language': 'en', 'score': 0.9999987929307774}
Er lebt mit seinen Eltern und seiner Schwester in Berlin. {'language': 'de', 'score': 0.999996045846908}
Yo me divierto todos los días en el parque. {'language': 'es', 'score': 0.9999960751128256}
Je m'appelle Angélica Summer, j'ai 12 ans et je suis canadienne. {'language': 'fr', 'score': 0.9999960488878062}


In [ ]:
from tqdm import tqdm 
random.shuffle(records)
for rec in tqdm(records):
    rec['Doc'] = nlp_model(rec['Text'])

100%|███████████████████████████████████████| 3252/3252 [05:13<00:00, 10.36it/s]


In [105]:
langDict = dict()
for rec in tqdm(records):
    for sent in rec['Doc'].sents:
        langDict.setdefault(sent._.language['language'], [])
        langDict[sent._.language['language']].append((sent, rec))

100%|███████████████████████████████████████| 3252/3252 [07:47<00:00,  6.96it/s]


In [106]:
langDict.keys()

dict_keys(['ru', 'UNKNOWN', 'bg', 'mk', 'uk', 'en', 'et', 'hr', 'tl', 'ro', 'lt', 'de', 'id', 'ca', 'sk'])

In [146]:
lg = 'sk'
print(len(langDict[lg]))
[print('Unique index:',d[1]['UniqueIndex'],'\n', d[0],'\n--') for d in random.sample(langDict[lg],1)]

1
Unique index: 1652 
 Како добро живети…
…Ako žiť dobre…
…Kako dobro živeti. 
--


[None]

In [151]:
# 523
r = [r for r in records if r['UniqueIndex'] == 3239][0]

In [157]:
r['Source'], r['Author'], r['Date posted']

('essentialpoetry', 'Линор Горалик', datetime.datetime(2018, 5, 9, 0, 0))

In [153]:
print(r['Text'])

Вот красным лесом красная лиса, – 
а он лежит, смешавшись с автоматом, 
в осеннем красном буром чернозёме, 
неглубоко, – 
и вот лиса несётся, 
пересекая сердце, горло, сердце, – 
подскакивает, лапой влезши в душу, 
отряхивает лапу, мчится дальше, – 
И он кричит распавшейся гортанью: 
 
КАКОГО ХУЯ, ГОСПОДИ, – ЗА ЧТО?! 
 
Я не успел – я инвалид по зренью, – 
я не успел, – они меня в апреле, 
когда уже исход и всё понятно, 
когда таких, как я, – едва одетых, 
полуслепых, хромых или безусых, – 
от киндер, кирхе, запаха из кюхен, – 
в зелёный их апрельский красный лес, 
где я от крови ничего не видел, 
и красный зверь, и горло, сердце, горло – 
а я ни разу даже не пальнул, 
я не успел – 
какого хуя, Боже?! 
 
ТАК ДАЙ МНЕ, ДАЙ МНЕ, ДАЙ МНЕ ЧТО-НИБУДЬ!!! 
 
И тут лиса упала и лежит.


In [133]:
# other

In [2]:
total = 0
for rec in records:
    total += len(str(rec['Text']).split(' '))
# 345043
total

342235

In [5]:
df['Source'].value_counts()

essentialpoetry      945
No War Poetry        724
metajournal          662
ROAR V3              350
Facebook             253
ROAR V2              234
Personal Telegram     54
Name: Source, dtype: int64

In [4]:
df['Before or after'].value_counts()

Before    1661
After     1561
Name: Before or after, dtype: int64

### Remove duplicates from corpus

In [222]:
print(random.choice(records)['Text'][:200])

Развивая Бродского
1
Я хотел бы жить, Иосиф, как можно дольше, пока
ритмично дыханье и тянется на бумаге строка,
жизнь - гостиный двор, мы временные постояльцы,
и какое мне дело до того городка,
где и


In [223]:
longStringCts = dict()
string2Recs = dict()
string2RecIdx = dict()

windowSize = 100
for i, rec in tqdm(enumerate(records)):
    text = rec['Text']
    rec['UniqueIndex'] = i
    if isinstance(text, str):
        for j in range(len(text)-windowSize):
            windowText = text[j:j+windowSize]
            longStringCts.setdefault(windowText, 0)
            longStringCts[windowText] += 1
            string2Recs.setdefault(windowText, [])
            string2Recs[windowText].append(rec)
            # string2RecIdx.setdefault(windowText, set())
            # string2RecIdx[windowText].add(i)

3502it [00:08, 406.86it/s] 


In [224]:
relevantCts = dict()
for s in tqdm(longStringCts):
    if longStringCts[s] > 1:
        relevantCts[s] = longStringCts[s]

100%|████████████████████████████| 2284719/2284719 [00:01<00:00, 1186974.80it/s]


In [225]:
overlappingRecs = []
for s in tqdm(relevantCts):
    shareString = string2Recs[s]
    if shareString not in overlappingRecs:
        # ids = [r['UniqueIndex'] for r in shareString]
        # if set(ids) 
        overlappingRecs.append(shareString)
len(overlappingRecs)

100%|█████████████████████████████████| 64074/64074 [00:00<00:00, 133374.94it/s]


228

In [143]:
import datetime

In [226]:
def decideRec(recList):
    # if just one, return rec
    if len(recList) == 1:
        return recList[0:]
    # convert to datetimes
    for r in recList:
        if not isinstance(r['Date posted'], datetime.datetime):
            # essentialpoetry      1023
            # No War Poetry         793
            # metajournal           713
            # ROAR V3               357
            # Facebook              281
            # ROAR V2               269
            # Personal Telegram      66
            if r['Source'] == 'ROAR V2':
                r['Date posted'] = datetime.datetime(2022, 6, 24, 0, 0)
            elif r['Source'] == 'ROAR V3':
                r['Date posted'] = datetime.datetime(2022, 8, 24, 0, 0)
            else:
                r['Date posted'] = datetime.datetime(2022,12,17,0,0)
    # compare date
    sortedByDate = []
    for i, rec in enumerate(recList):
        sortedByDate.append((rec['Date posted'], i))
    sortedByDate = sorted(sortedByDate)
    # check types
    if isinstance(sortedByDate[0], datetime.datetime) and isinstance(sortedByDate[1], datetime.datetime):
        if sortedByDate[0] != sortedByDate[1]:
            return recList[sortedByDate[0][1]]
    else:
        idxs = [s[1] for s in sortedByDate[1:]]
        return [recList[j] for j in idxs]
    
    # compare length of text
    sortedByLen = []
    for i, rec in enumerate(recList):
        sortedByLen.append((len(rec['Text']), i))
    sortedByLen = sorted(sortedByLen)
    idxs = [s[1] for s in sortedByLen[1:]]
    return [recList[j] for j in idxs]

In [229]:
records[162]

{'Text': '\nРевёт жена опальная,\nв правах поражена.\nИдёт война двуспальная,\nгражданская война.\nПомыты кухня, ванная\nслезами в три ручья.\nИдёт война диванная,\nтрещит по швам семья.\n+\nБудущему малышу,\nптицам небесным пища?\nСижу в уголке, пишу\nугольком пепелища\nродного. Роднее нет.\nМай. Четвёртое мая.\nВ окне кабинета свет:\nне выключила, убегая.\n+\nВсе страны, кроме одной.\nЧто же мы так бездомны?\nВстречаемся, мой родной,\nоколо пятой колонны\nбольшого театра войны —\nв Эривани, в Тифлисе,\nкак рыбе зонтик нужны\nмировой закулисе.\n+\nОт грохота, скрежета, лязга\nсжимается матка.\nКачает пустую коляску.\nЦе хлопчик? Дiвчатко.\nПоёт колыбельную дочке.\nПечаль непроглядна.\nВойна. Молока на сорочке\nрасплывающиеся пятна.\n+\nСвидетельств — терабайт,\nкак мой народ ушёл\nв искусственный офсайд.\nНо засчитали гол\nи торжествует зло,\nи стадион орёт.\nМне очень повезло\nуспеть на самолёт.\n+\nМеждународный язык —\nмладенческий лепет.\nСпит у груди призывник\nГолгофы. Колеблет\

In [230]:
counts = dict()
for recList in overlappingRecs:
    counts.setdefault(len(recList),0)
    counts[len(recList)] += 1

In [231]:
# create new list using only the earliest overlapping rec
repeatIdxs = set()
for recList in overlappingRecs:
    badRecs = decideRec(recList)
    badIdxs = [b['UniqueIndex'] for b in badRecs]
    for b in badIdxs:
        repeatIdxs.add(b)

In [232]:
cleanRecList = []
j = 0
for i, rec in enumerate(records):
    if rec['UniqueIndex'] not in repeatIdxs:
        rec['UniqueIndex'] = j
        cleanRecList.append(rec)
        j += 1
len(cleanRecList)

3289

In [234]:
cleanRecList[1000]

{'Text': '***\n \n– В Москве – невкусная клубника, бери водку!\n \nВ вытянутой трубке сна так тихо,\n \nЧто можно услышать\n \nНеуловимую чёрную лодку.\n \nСпрашиваю:\n \n– Это каяк?\n \nПтица с ветки:\n \n– Каяк, каяк. Что с тобою не так?\n \n– Со мною не так носовой платок,\n \nКаждое слово.\n \nНеба бесформенный носорог\n \nОживает в половине шестого.\n \nНаблюдаю за ним из окна.\n \nУ меня проблема только одна:\n \n«Скендербеу», «Хабнарфьордюр» и «Нымме Калью» проиграли свои матчи.\n \nИ три любимых, но непростых занятья:\n \nСмотреть на осину,\nСмотреть на берёзу,\nСмотреть на тополь.\n \nМошка умерла у меня на глазах,\nА что пережил ты?\n \n \n(интернет-журнал ”Literratura”)\n \n#выбор_Максима_Дрёмова',
 'Author': 'Денис Крюков',
 'Before or after': 'Before',
 'Source': 'metajournal',
 'Date posted': datetime.datetime(2019, 11, 14, 0, 0),
 'UniqueIndex': 1000}

In [233]:
cleanDf = pd.DataFrame.from_records(cleanRecList)

In [235]:
cleanDf.to_excel('../Excel_files/Full_Poem_Dataset_12-17.xlsx')

## AUTHOR DATA

In [7]:
authorInfo = pd.read_excel('../Excel_files/Thesis Authors.xlsx')
authorInfo.head()

,Number of Poems,Author,City,Country,Identification,Notes,Link to bio
0,3,Виген Аракелян,Razdan,Armenia,NaN,NaN,https://polutona.ru/?show=arakelian
1,1,Герман Лукомников,Baku,Azerbaijan,NaN,NaN,https://ru.wikipedia.org/wiki/%D0%9B%D1%83%D0%...
2,1,Александр Иличевский,Sumgait,Azerbaijan,NaN,NaN,https://ru.wikipedia.org/wiki/%D0%98%D0%BB%D0%...
3,7,Леонид Шваб,Babruysk,Belarus,NaN,NaN,http://www.litkarta.ru/world/israel/persons/sh...
4,6,Ирина Валерина,Babruysk,Belarus,NaN,NaN,NaN


In [9]:
authorInfo['Country'].value_counts()

Russia          370
Ukraine          86
Kazakhstan       26
Belarus          16
Uzbekistan        6
Moldova           5
Latvia            5
Estonia           4
Kyrgyzstan        2
Azerbaijan        2
Lithuania         1
Iran              1
Georgia           1
Tajikistan        1
Turkmenistan      1
USA               1
Armenia           1
Name: Country, dtype: int64

In [ ]:
value_counts